# Calculate relevant variable

In [ ]:
import set_path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from shapely.geometry import MultiLineString, MultiPoint
from shapely.ops import linemerge, nearest_points
import geopandas as gpd
from geopandas import GeoDataFrame
from centerline.geometry import Centerline

import networkx as nx
import momepy

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import upc_sw.poly_utils as poly_utils

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

## Settings

In [ ]:
# Paths
bgt_road_file = '../datasets/bgt/bgt_voetpad.gpkg'
point_cloud_file = '../datasets/output/sidewalks_with_obstacles.gpkg'
segments_file = '../datasets/output/sidewalk_segments.gpkg'
output_file = '../datasets/output/final_output_segments.geojson'
output_image = '../datasets/output/final_output_image.png'

# A CRS tells Python how those coordinates relate to places on the Earth. Rijksdriehoek = epsg:28992
CRS = 'epsg:28992'

# Boundary for filtering out (in meters)
min_path_width = 0.4 

# Boundaries between the final colors green/orange/red (in meters)
width_lower = 0.9  
width_upper = 1.5 

# Maximum distance between intended start point and start node (in meters)
max_dist = 3 

# Maximum length of linestring (in meters), otherwise cut
max_ls_length = 30

## Import data

In [ ]:
# Read BGT data
df_bgt = gpd.read_file(bgt_road_file, crs=CRS)

In [ ]:
# Read sidewalk with obstacle data (point cloud)
df_pc = gpd.read_file(point_cloud_file, crs=CRS)

In [ ]:
# Read lines with widths (calculated in previous notebook)
df_segments = gpd.read_file(segments_file, crs=CRS)

## Process data

### Remove too narrow paths

In [ ]:
# Apply minimal path width
df_segments_wide = df_segments[df_segments['min_width'] > min_path_width].reset_index(drop=True)
print(df_segments.shape)
print(df_segments_wide.shape)

In [ ]:
# Remove short dead-end segments - need to make multilinestring first to make it work properly
long_segments = poly_utils.remove_short_lines(MultiLineString(list(df_segments_wide['geometry']))) # slow
long_segments_df = GeoDataFrame(long_segments, columns = ['geometry'])
df_segments_wide = df_segments_wide.merge(long_segments_df, how='inner')
df_segments_wide.shape

### Use color codes

In [ ]:
conditions = [
    (df_segments_wide['min_width'] < width_lower),
    (df_segments_wide['min_width'] >= width_lower) & (df_segments_wide['min_width'] < width_upper),
    (df_segments_wide['min_width'] >= width_upper)
]

values = ['red', 'orange', 'lightgreen']

In [ ]:
df_segments_wide['min_width_color'] = np.select(conditions, values)
df_segments_wide['min_width_color'].value_counts()

In [ ]:
# Add width factor, for calculating the weights of the paths later
values = [1000000, 1000, 1]
df_segments_wide['min_width_factor'] = np.select(conditions, values)

### Create centerlines without obstacles

In [ ]:
# Put polygons together
gdf_bgt = GeoDataFrame(geometry=gpd.GeoSeries(df_bgt['geometry'].unary_union))
gdf_bgt = gpd.GeoDataFrame(gdf_bgt.geometry.explode()) 
#gdf_bgt = GeoDataFrame(geometry=gpd.GeoSeries(df_bgt['geometry'])) # if you don't want to merge sidewalk polygons

In [ ]:
# Calculate centerlines
gdf_bgt['centerlines'] = gdf_bgt.progress_apply(
    lambda row: Centerline(row.geometry, interpolation_distance=0.5), axis=1)
gdf_bgt = gdf_bgt.set_geometry('centerlines')

#### Remove dead-ends

In [ ]:
gdf_bgt['centerlines'] = gdf_bgt['centerlines'].progress_apply(linemerge)

In [ ]:
gdf_bgt['centerlines'] = gdf_bgt['centerlines'].progress_apply(poly_utils.remove_short_lines)

## Create relevant variable

In [ ]:
# Create final dataframe
final_df = pd.DataFrame()

for i in range(len(gdf_bgt['centerlines'])):
    
    # Get centerline
    my_centerline = gdf_bgt['centerlines'].values[i]
    
    # Create dataframe with linestrings of centerline
    centerline_df = poly_utils.create_df_centerlines(my_centerline)
    centerline_df['length'] = centerline_df['geometry'].length
    centerline_df['route_weight'] = np.nan
    
    # Cut linestrings that are too long
    centerline_df = poly_utils.shorten_linestrings(centerline_df, max_ls_length)
        
    # Get sidewalk polygon for this centerline
    my_sidewalk = gdf_bgt['geometry'].values[i]
    
    # Create graph for all paths withing this sidewalk polygon
    df_sidewalk = df_segments_wide[df_segments_wide['geometry'].within(my_sidewalk)].reset_index(drop=True)
    G = momepy.gdf_to_nx(df_sidewalk, approach="primal", multigraph=True)
    
    for j in range(len(centerline_df['geometry'])):
        
        # Get line
        my_line = centerline_df.iloc[[j]]['geometry'].values[0]
        
        # Get origin and destination location
        if len(list(G.nodes)) > 0:
            origin_point, dest_point = my_line.boundary
            origin_node_loc = nearest_points(origin_point, MultiPoint(list(G.nodes)))[1]
            dest_node_loc = nearest_points(dest_point, MultiPoint(list(G.nodes)))[1]

            # Get origin and destination node
            if (origin_point.distance(origin_node_loc) > max_dist) or (dest_point.distance(dest_node_loc) > max_dist):
                print('origin and/or destination node too far from line start/end for line (j)', j, 'in sidewalk (i)', i) 
            else:    
                origin_node = (origin_node_loc.x, origin_node_loc.y)
                dest_node = (dest_node_loc.x, dest_node_loc.y)   

                # Get weight of optimal route in graph
                try:
                    route_weight = nx.shortest_path_length(G, origin_node, dest_node, 
                                                           weight='min_width_factor')
                    centerline_df['route_weight'][j] = route_weight
                except nx.NetworkXNoPath:
                    print('no route found for line (j)', j, 'in sidewalk (i)', i)       
        else:
            print('network has zero nodes')

    # Append data to final dataframe
    final_df = final_df.append(centerline_df)
final_df = final_df.reset_index()    

In [ ]:
# Get final color of the routes
final_df['final_color'] = final_df.progress_apply(
    lambda row: poly_utils.get_route_color(row.route_weight), axis=1)

In [ ]:
# Get dataframe with only valid lines
final_df_select = final_df[final_df['final_color'].isin(['lightgreen', 'orange', 'red'])]

## Visualisation

In [ ]:
# Boundaries for plotting a subset of the data - select part of dataframe before plotting instead?
#x_min = 125730 # area 4 
#x_max = 125810
#y_min = 489820
#y_max = 489960
#x_min = 122700 # area 13
#x_max = 122800
#y_min = 490240
#y_max = 490340
#x_min = 114680 # area 8
#x_max = 114830
#y_min = 487520
#y_max = 487690
x_min = 122400 # larger area east
x_max = 123600
y_min = 485400
y_max = 486100

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,5), dpi=100, 
                                    frameon=False, constrained_layout=True)

# Plot process step 1
df_bgt.plot(ax=ax1, color="silver")
df_pc.plot(ax=ax1)
df_segments_wide.plot(ax=ax1, color=df_segments_wide['min_width_color'])
ax1.axis('off')

# Plot process step 2
df_bgt.plot(ax=ax2, color="silver")
df_pc.plot(ax=ax2)
final_df.plot(ax=ax2, linewidth=3, color=final_df.final_color)  
ax2.axis('off')

# Plot process step 3
df_bgt.plot(ax=ax3)
final_df_select.plot(ax=ax3, linewidth=3, color=final_df_select.final_color)  
ax3.axis('off')

# Create legend
acc = mpatches.Patch(color='lightgreen', label='accessible (>' + str(width_upper) + 'm)')
narrow = mpatches.Patch(color='orange', label='narrow (' + str(width_lower) + '-' + str(width_upper) + 'm)')
notacc = mpatches.Patch(color='red', label='not accessible (<' + str(width_lower) + 'm)')
rnan = mpatches.Patch(color='purple', label='no route (node too far or no path)')
rl0 = mpatches.Patch(color='black', label='route length 0')
plt.legend(handles=[acc,narrow,notacc,rl0,rnan], 
           bbox_to_anchor=(0.5, -0.5, 0.5, 0.5))

# Set plot limits
ax1.set_xlim([x_min, x_max])   
ax1.set_ylim([y_min, y_max])
ax2.set_xlim([x_min, x_max])
ax2.set_ylim([y_min, y_max])
ax3.set_xlim([x_min, x_max])
ax3.set_ylim([y_min, y_max])

plt.savefig(output_image, bbox_inches='tight')
plt.show()

## Post-process output

In [ ]:
final_df = final_df.set_crs(CRS) 
df_projected = final_df #.to_crs('epsg:4326')  # TODO: check CRS they want for maps.amsterdam.nl

In [ ]:
# TODO: any post-processing we need to do for maps.amsterdam.nl?

## Store output

In [ ]:
with open(output_file, 'w') as f:
    f.write(df_projected.to_json())